# Observer Pattern

The observer pattern is a software design pattern in which an object, called the subject, maintains a list of its dependents, called observers, and notifies them of state changes, typically by calling one of their methods. It is mainly used to implement distributed event handling systems.

In [2]:
use std::sync::{Arc, Weak};

pub trait Observer {
    type Subject;

    fn update(&self, subject: &Self::Subject);
}

pub trait Observable {
    type ObserverPtr;

    fn add_observer(&mut self, observer: Self::ObserverPtr);
    fn remove_observer(&mut self, observer: Self::ObserverPtr);
    fn notify(&self);
}

In [3]:
pub struct Subject {
    observers: Vec<Weak<dyn Observer<Subject = Self>>>,
    state: String,
}

impl Observable for Subject {
    type ObserverPtr = Arc<dyn Observer<Subject = Self>>;

    fn add_observer(&mut self, observer: Self::ObserverPtr) {
        self.observers.push(Arc::downgrade(&observer));
    }

    fn remove_observer(&mut self, observer: Self::ObserverPtr) {
        self.observers
            .retain(|o| !o.ptr_eq(&Arc::downgrade(&observer)));
    }

    fn notify(&self) {
        self.observers
            .iter()
            .flat_map(|o| o.upgrade())
            .for_each(|o| o.update(&self));
    }
}

impl Subject {
    fn new(state: &str) -> Self {
        Self {
            observers: Vec::new(),
            state: state.into(),
        }
    }

    fn set_state(&mut self, state: &str) {
        if self.state == state {
            return;
        }

        self.state = state.into();
        self.notify();
    }

    fn state(&self) -> &str {
        &self.state
    }
}

In [4]:
struct ConcreteObserver {
    name: String,
}

impl ConcreteObserver {
    fn new(name: &str) -> Arc<Self> {
        Arc::new(Self { name: name.into() })
    }
}

impl Observer for ConcreteObserver {
    type Subject = Subject;

    fn update(&self, subject: &Self::Subject) {
        println!("Observing subject with state={:?} in {}", subject.state(), self.name);
    }
}

In [5]:
fn main() {
    let mut subject = Subject::new("initial state");

    let observer1 = ConcreteObserver::new("observer1");
    let observer2 = ConcreteObserver::new("observer2");

    subject.add_observer(observer1.clone());
    subject.add_observer(observer2.clone());

    subject.set_state("new state - A");
    subject.set_state("new state - B");

    subject.remove_observer(observer1);

    subject.set_state("new state - C");
    subject.set_state("new state - D");
}

main();

Observing subject with state="new state - A" in observer1
Observing subject with state="new state - A" in observer2
Observing subject with state="new state - B" in observer1
Observing subject with state="new state - B" in observer2
Observing subject with state="new state - C" in observer2
Observing subject with state="new state - D" in observer2


# 

# Callbacks

In [7]:
type BoxedCallback<TResult = ()> = Box<dyn Fn() -> TResult>;
type BoxedCallbackWithArg<T, TResult = ()> = Box<dyn Fn(T) -> TResult>;

In [12]:
struct Printer {
    id: String,
    on_start: Option<BoxedCallback>,
    on_print: Option<BoxedCallbackWithArg<String>>,
    on_stop: Option<BoxedCallback>,
}

impl Printer {
    fn new(id: &str) -> Self {
        Self {
            id: id.into(),
            on_start: None,
            on_print: None,
            on_stop: None,
        }
    }

    fn on_start(&mut self, callback: BoxedCallback) {
        self.on_start = Some(callback);
    }

    fn on_print(&mut self, callback: BoxedCallbackWithArg<String>) {
        self.on_print = Some(callback);
    }

    fn on_stop(&mut self, callback: BoxedCallback) {
        self.on_stop = Some(callback);
    }

    fn start(&self) {
        if let Some(ref on_start) = self.on_start {
            on_start();
        }

        println!("Printer {} started", self.id);
    }

    fn print(&self, text: &str) {
        self.start();

        if let Some(ref on_print) = self.on_print {
            on_print(text.into());
        }

        println!("Printer {} printed: {}", self.id, text);

        self.stop();
    }

    fn stop(&self) {
        if let Some(ref on_stop) = self.on_stop {
            on_stop();
        }

        println!("Printer {} finished", self.id);
    }
}

In [15]:
let mut printer = Printer::new("PRN_1");

printer.on_start(Box::new(|| {
    println!("Log: Printer started");
}));

printer.on_print(Box::new(|text| {
    println!("Log: Printing - {}", text);
}));

printer.on_stop(Box::new(|| {
    println!("Log: Printer stopped");
}));

printer.print("Hello, World!");

Log: Printer started
Printer PRN_1 started
Log: Printing - Hello, World!
Printer PRN_1 printed: Hello, World!
Log: Printer stopped
Printer PRN_1 finished
